In [1]:
#!pip install tensorflow==2.5.0
import math
from datetime import datetime
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, accuracy_score, recall_score, precision_score, f1_score
import random as python_random

!pip install image-classifiers==1.0.0b1
!pip install keras-applications
from classification_models.tfkeras import Classifiers

  Preparing metadata (setup.py) ... done
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0b1-py3-none-any.whl size=19954 sha256=acb588c4e7f1fe8bd195b0b812c2197a5ee493eaee8dc07a938f036fd79cb888
  Stored in directory: /root/.cache/pip/wheels/bd/0d/8f/a84087f2ce6e91102d92816969d7cfe6179e6e3832199bf63b
Successfully built image-classifiers


### Retinal Scans - Fundus Model

In [2]:
np.random.seed(2021)
python_random.seed(2021)
tf.random.set_seed(2021)

In [3]:
mirrored_strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(mirrored_strategy.num_replicas_in_sync))

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1
The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [5]:
import os
ls_A = os.listdir("datasets/rop_seg_data/train/white")
ls_B = os.listdir("datasets/rop_seg_data/train/black")

In [6]:
len(ls_A)
ls_A.sort()

In [7]:
len(ls_B)
ls_B.sort()

In [8]:
data = []
for i in range(len(ls_A)):
    data.append(["white/"+ls_A[i], "WHITE"])
    
for i in range(len(ls_B)):
    data.append(["black/"+ls_B[i], "BLACK/AFRICAN AMERICAN"])

In [9]:
combined = pd.DataFrame(data, columns=["image_id", "race"])

In [10]:
combined.to_csv("rop_data_fundus.csv")

In [11]:
data_df = pd.read_csv('rop_data_fundus.csv')
data_df = data_df[data_df.race.isin(['WHITE','BLACK/AFRICAN AMERICAN'])]

In [12]:
data_df = data_df.sample(frac=1).reset_index(drop=True)

In [14]:
train_df = data_df[:int(len(data_df)*8/10)]
valid_df = data_df[int(len(data_df)*8/10):]

In [15]:
valid_df.race.value_counts(normalize=True)

WHITE                     0.627139
BLACK/AFRICAN AMERICAN    0.372861
Name: race, dtype: float64

In [16]:
len(valid_df)

818

In [17]:
HEIGHT, WIDTH = 256, 256

arc_name = "ROP-" + str(HEIGHT) + "x" + str(WIDTH) + "80-20-split-resnet18-Float16_2-race_detection_rop_seg_data_fundus"

In [18]:
resnet18, preprocess_input = Classifiers.get('resnet18')

In [19]:
with mirrored_strategy.scope():
    input_a = Input(shape=(HEIGHT, WIDTH, 3))
    base_model = resnet18(input_tensor=input_a, include_top=False, input_shape=(HEIGHT,WIDTH,3), weights='imagenet')
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(2, name='dense_logits')(x)
    output = Activation('softmax', dtype='float32', name='predictions')(x)
    model = Model(inputs=[input_a], outputs=[output])

44933120/44920640 [==============================] - 3s 0us/step


In [20]:
tf.__version__

'2.6.2'

In [21]:
learning_rate = 1e-3
momentum_val=0.9
decay_val= 0.0
batch_s = 128 # may need to reduce batch size if OOM error occurs
train_batch_size = batch_s
test_batch_size = 128

reduce_lr = ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.1, patience=2, min_lr=1e-5, verbose=1)

adam_opt = tf.keras.optimizers.Adam(learning_rate=learning_rate, decay=decay_val)
adam_opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(adam_opt, "dynamic")


with mirrored_strategy.scope():
    model.compile(optimizer=adam_opt,
                    loss=tf.losses.CategoricalCrossentropy(),
                    metrics=[
                        tf.keras.metrics.AUC(curve='ROC', name='ROC-AUC'),
                        tf.keras.metrics.AUC(curve='PR', name='PR-AUC')
                    ],
    )

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)


In [22]:
train_gen = ImageDataGenerator(
            rotation_range=15,
            fill_mode='constant',
            horizontal_flip=True,
            zoom_range=0.1,
            preprocessing_function=preprocess_input
            )

validate_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [23]:
train_batches = train_gen.flow_from_dataframe(train_df, directory="datasets/rop_seg_data/train", x_col="image_id", y_col="race", class_mode="categorical",target_size=(HEIGHT, WIDTH),shuffle=True,seed=2021,batch_size=train_batch_size, dtype='float32')
validate_batches = validate_gen.flow_from_dataframe(valid_df, directory="datasets/rop_seg_data/train", x_col="image_id", y_col="race", class_mode="categorical",target_size=(HEIGHT, WIDTH),shuffle=False,batch_size=test_batch_size, dtype='float32')        

Found 3268 validated image filenames belonging to 2 classes.
Found 818 validated image filenames belonging to 2 classes.


In [24]:
train_batches

In [25]:
train_epoch = math.ceil(len(data_df) / train_batch_size)
val_epoch = math.ceil(len(valid_df) / test_batch_size)

In [26]:
var_date = datetime.now().strftime("%Y%m%d-%H%M%S")
ES = EarlyStopping(monitor='val_loss', mode='min', patience=4, restore_best_weights=True)
checkloss = ModelCheckpoint("saved_models/" + str(arc_name) + "_rop_seg-" + str(learning_rate) + "_" + var_date+"_epoch:{epoch:03d}.hdf5", monitor='val_loss', mode='min', verbose=1, save_best_only=True, save_weights_only=False)

In [ ]:
model.fit(train_batches,
          validation_data = validate_batches,
            epochs=100,
            steps_per_epoch=int(train_epoch),
            workers=32,
            max_queue_size=50,
            shuffle=True,
            callbacks=[checkloss, reduce_lr, ES]
           )

Epoch 1/100
28/32 [=========================>....] - ETA: 58:29 - loss: 0.3453 - ROC-AUC: 0.9478 - PR-AUC: 0.9408  

## Chest Scans

In [ ]:
from collections import Counter
counter = Counter(train_batches.classes)                          
max_val = float(max(counter.values()))       
train_class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}

In [ ]:
train_class_weights

In [ ]:
validate_batches = validate_gen.flow_from_dataframe(validation_df, directory="../Public_Datasets/mimic-cxr-jpg/physionet.org/files/mimic-cxr-jpg/2.0.0/files/", x_col="path", y_col="race", class_mode="categorical",target_size=(HEIGHT, WIDTH),shuffle=False,batch_size=test_batch_size, dtype='float32')

In [ ]:
var_date = datetime.now().strftime("%Y%m%d-%H%M%S")
ES = EarlyStopping(monitor='val_loss', mode='min', patience=4, restore_best_weights=True)
checkloss = ModelCheckpoint("saved_models/" + str(arc_name) + "_CXR_LR-" + str(learning_rate) + "_" + var_date+"_epoch:{epoch:03d}_val_loss:{val_loss:.5f}.hdf5", monitor='val_loss', mode='min', verbose=1, save_best_only=True, save_weights_only=False)

In [ ]:
model.fit(train_batches,
            validation_data=validate_batches,
            epochs=100, class_weight = train_class_weights,
            steps_per_epoch=int(train_epoch),
            validation_steps=int(val_epoch),
            workers=32,
            max_queue_size=50,
            shuffle=True,
            callbacks=[checkloss, reduce_lr, ES]
           )

In [ ]:
test_df.race.value_counts(normalize=True)

In [ ]:
test_model = "saved_models/MIMIC-320x32060-10-30-split-resnet-Float16_3-race_detection_lowpass25_CXR_LR-0.001_20211103-193808_epoch:007_val_loss:0.24422.hdf5"

In [ ]:
model = tf.keras.models.load_model(test_model)

In [ ]:
test_batches = validate_gen.flow_from_dataframe(test_df, directory = "Lowpass25_Images/",x_col="path", y_col="race", class_mode="categorical",target_size=(HEIGHT, WIDTH),shuffle=False,batch_size=128, dtype='float32', interpolation='bilinear') 

In [ ]:
with mirrored_strategy.scope():
    multilabel_predict_test = model.predict(test_batches, max_queue_size=10, verbose=1, steps=math.ceil(len(test_df)/128), workers=16)

In [ ]:
multilabel_predict_test.shape

In [ ]:
input_prediction = multilabel_predict_test
input_df = test_df
input_prediction_df = pd.DataFrame(input_prediction)
true_logits = pd.DataFrame()
loss_log = pd.DataFrame()

In [ ]:
input_prediction_df.shape

In [ ]:
def stat_calc(input_prediction_df, input_df):
    ground_truth = input_df.race
    pathology_array=['ASIAN',
        'BLACK/AFRICAN AMERICAN',
        'WHITE'
        ]
    i=0
    auc_array = []
    for pathology in pathology_array:
        new_truth = (ground_truth.str.contains(pathology)).apply(int)
        print(new_truth.shape)
        input_prediction_val = input_prediction_df[i]
        val = input_prediction_val
        AUC = roc_auc_score(new_truth, val)
        #AUC = average_precision_score(new_truth, val, average=None)
        true_logits.insert(i, i, new_truth, True)
        auc_array.append(AUC)
        i += 1
        
    progress_df = pd.DataFrame({'Study':pathology_array, 'AUC':auc_array})
    print(progress_df)

In [ ]:
stat_calc(input_prediction_df, input_df)

In [ ]:
#test_model = "saved_models/MIMIC-256x25660-10-30-split-resnet-Float16_3-race_detection_CXR_LR-0.001_20211021-052440_epoch:016_val_loss:0.24378.hdf5"
test_model = "saved_models/MIMIC-256x25660-10-30-split-resnet-Float16_3-race_detection_CXR_LR-0.001_20211021-052440_epoch:016_val_loss:0.24378.hdf5"

In [ ]:
model = tf.keras.models.load_model(test_model)

In [ ]:
import os
ls = os.listdir("pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_23/images/")

In [ ]:
ls.sort()

In [ ]:
data = []
for i in range(len(ls)//6):
    data.append([ls[6*i], "WHITE"])
    data.append([ls[6*i+1], "BLACK/AFRICAN AMERICAN"])
    data.append([ls[6*i+2], "WHITE"])
    data.append([ls[6*i+3], "BLACK/AFRICAN AMERICAN"])
    data.append([ls[6*i+4], "WHITE"])
    data.append([ls[6*i+5], "BLACK/AFRICAN AMERICAN"])

In [ ]:
data[48]

In [ ]:
len(data)

In [ ]:
combined = pd.DataFrame(data, columns=["path", "race"])

In [ ]:
combined.race.value_counts(normalize=True)

In [ ]:
test_batches = validate_gen.flow_from_dataframe(combined, directory = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_25/images/",x_col="path", y_col="race", class_mode="categorical",target_size=(HEIGHT, WIDTH),shuffle=False,batch_size=128, dtype='float32', interpolation='bilinear')

In [ ]:
with mirrored_strategy.scope():
    multilabel_predict_test = model.predict(test_batches, max_queue_size=10, verbose=1, steps=math.ceil(len(combined)/128), workers=16)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
combined.shape

In [ ]:
multilabel_predict_test.shape

In [ ]:
result = multilabel_predict_test
#result = model.predict(validate_batches, val_epoch)
labels = np.argmax(result, axis=1)
target_names = ['Asian','Black', 'White']

In [ ]:
l = []

for i in range(len(labels)//6):
    l.append(labels[6*i:6*(i+1)])

In [ ]:
predict_correct = []
predict_incorrect = []
for i in range(len(l)):
    if np.array_equal([1,2,2,1,2,1], l[i]):
        predict_correct.append(i)
    else:
        predict_incorrect.append(i)

In [ ]:
black_probs = []
white_probs = []
for i in predict_correct:
    white_probs.append(result[6*i][1])
    black_probs.append(result[6*i+1][2])

In [ ]:
len(black_probs)

In [ ]:
predict_incorrect

In [ ]:
probs = {}
for i in range(len(result)):
    probs[i] = result[i]

In [ ]:
probs

In [ ]:
ls[6*25]

In [ ]:
probs[6*27]

In [ ]:
epochs= [15,18,20,23,25]
white_probs = [3.8162336e-01, 1.2340636e-01, 6.4106297e-01, 6.5219790e-01, 7.1337724e-01]
fakeA_white = [6.1822724e-01, 8.7649578e-01, 3.5890210e-01,  3.4773538e-01, 2.8654653e-01]
fakeB_black = [2.4298365e-04, 2.9653276e-04,3.8822962e-04,2.2254302e-04, 1.8199782e-04]
black_probs = [9.9974924e-01, 9.9969411e-01,9.9960214e-01,  9.9976617e-01, 9.9981230e-01]

plt.figure(figsize=(10,8))
plt.plot(epochs, white_probs)
plt.plot(epochs, fakeA_white)
plt.plot(epochs, black_probs)
plt.plot(epochs, fakeB_black)

plt.legend(["fake_A predicted as Black", "fake_A predicted as White","fake_B predicted as White","fake_B predicted as Black"])
plt.title("Probabilities - Image (9c5e78e3-ac01ec60-ab719485-03063374-c488be42)")

In [ ]:
predict_incorrect

In [ ]:
import cv2
path_fakeA = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_18/images/9c5e78e3-ac01ec60-ab719485-03063374-c488be42_fake_A.png"
path_fakeB = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_18/images/9c5e78e3-ac01ec60-ab719485-03063374-c488be42_fake_B.png"
path_realA = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_18/images/9c5e78e3-ac01ec60-ab719485-03063374-c488be42_real_A.png"
path_realB = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_18/images/9c5e78e3-ac01ec60-ab719485-03063374-c488be42_real_B.png"

path_fakeA2 = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_18/images/9c5e78e3-ac01ec60-ab719485-03063374-c488be42_fake_A.png"
path_fakeB2 = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_18/images/9c5e78e3-ac01ec60-ab719485-03063374-c488be42_fake_B.png"
path_realA2 = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_18/images/9c5e78e3-ac01ec60-ab719485-03063374-c488be42_real_A.png"
path_realB2 = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_18/images/9c5e78e3-ac01ec60-ab719485-03063374-c488be42_real_B.png"

img_fakeA = cv2.imread(path_fakeA)
img_fakeB = cv2.imread(path_fakeB)
img_realA = cv2.imread(path_realA)
img_realB = cv2.imread(path_realB)

img_fakeA2 = cv2.imread(path_fakeA2)
img_fakeB2 = cv2.imread(path_fakeB2)
img_realA2 = cv2.imread(path_realA2)
img_realB2 = cv2.imread(path_realB2)

In [ ]:
diff_farb = (img_fakeA-img_realB)
diff_fbra = (img_fakeB-img_realA)

diff_farb2 = (img_fakeA2-img_realB2)
diff_fbra2 = (img_fakeB2-img_realA2)

In [ ]:
#image = img[...,0]
from PIL import Image, ImageFilter
plt.figure(figsize=(20,15))
plt.suptitle("Image-9c5e78e3-ac01ec60-ab719485-03063374-c488be42")
#
plt.subplot(341),plt.imshow(img_fakeA, cmap = 'gray')
plt.title('Fake_A image - Epoch18'), plt.xticks([]), plt.yticks([])
plt.subplot(342),plt.imshow(img_fakeB, cmap = 'gray')
plt.title('Fake_B image - Epoch18'), plt.xticks([]), plt.yticks([])
plt.subplot(345),plt.imshow(img_realB, cmap = 'gray')
plt.title('Real_B image - Epoch18'), plt.xticks([]), plt.yticks([])
plt.subplot(346),plt.imshow(img_realA, cmap = 'gray')
plt.title('Real_A image - Epoch18'), plt.xticks([]), plt.yticks([])
plt.subplot(349),plt.imshow(diff_farb, cmap = 'gray')
plt.title('fakeA-realB - Epoch18'), plt.xticks([]), plt.yticks([])
plt.subplot(3,4,10),plt.imshow(diff_fbra, cmap = 'gray')
plt.title('fakeB-realA - Epoch18'), plt.xticks([]), plt.yticks([])

plt.subplot(3,4,3),plt.imshow(img_fakeA2, cmap = 'gray')
plt.title('Fake_A image - Epoch25'), plt.xticks([]), plt.yticks([])
plt.subplot(344),plt.imshow(img_fakeB2, cmap = 'gray')
plt.title('Fake_B image - Epoch25'), plt.xticks([]), plt.yticks([])
plt.subplot(347),plt.imshow(img_realB2, cmap = 'gray')
plt.title('Real_B image - Epoch25'), plt.xticks([]), plt.yticks([])
plt.subplot(348),plt.imshow(img_realA2, cmap = 'gray')
plt.title('Real_A image - Epoch25'), plt.xticks([]), plt.yticks([])
plt.subplot(3,4,11),plt.imshow(diff_farb2, cmap = 'gray')
plt.title('fakeA-realB - Epoch25'), plt.xticks([]), plt.yticks([])
plt.subplot(3,4,12),plt.imshow(diff_fbra2, cmap = 'gray')
plt.title('fakeB-realA - Epoch25'), plt.xticks([]), plt.yticks([])
#
plt.show()

In [ ]:
import cv2
path_fakeA = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_15_50img/images/001ccf5a-808a286a-b93f7df2-d12167ad-103d015e_fake_A.png"
path_fakeB = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_15_50img/images/001ccf5a-808a286a-b93f7df2-d12167ad-103d015e_fake_B.png"
path_realA = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_15_50img/images/001ccf5a-808a286a-b93f7df2-d12167ad-103d015e_real_A.png"
path_realB = "pytorch-CycleGAN-and-pix2pix-master/results/white_black/test_15_50img/images/001ccf5a-808a286a-b93f7df2-d12167ad-103d015e_real_B.png"

img_fakeA = cv2.imread(path_fakeA, cv2.IMREAD_GRAYSCALE)
img_fakeB = cv2.imread(path_fakeB, cv2.IMREAD_GRAYSCALE)
img_realA = cv2.imread(path_realA, cv2.IMREAD_GRAYSCALE)
img_realB = cv2.imread(path_realB,cv2.IMREAD_GRAYSCALE)

In [ ]:
z = (x==y)
f = z[z==False]

In [ ]:
len(f)/(256*256)

In [ ]:
image1= np.int32(img_fakeA)

image2= np.int32(img_realB)

diff_farb = image1 - image2

In [ ]:
image1= np.int32(img_fakeB)

image2= np.int32(img_realA)

diff_fbra = image1 - image2

In [ ]:
diff_farb = cv2.subtract(img_fakeA,img_realB) # #img_fakeA - img_realB #
diff_fbra = img_fakeB - img_realA #cv2.subtract(img_fakeB,img_realA) # #

In [ ]:
h = diff_farb[diff_farb <0]

In [ ]:
image1= np.int32(img_fakeA)

image2= np.int32(img_realB)

diff_farb = image1 - image2

In [ ]:
#image = img[...,0]
from PIL import Image, ImageFilter
plt.figure(figsize=(10,15))
#plt.suptitle("25th Epoch (Image-001ccf5a-808a286a-b93f7df2-d12167ad-103d015e) With only one channel in case of difference images")
#
plt.subplot(321),plt.imshow(img_fakeA, cmap = 'gray')
plt.title('Fake_A image'), plt.xticks([]), plt.yticks([])
plt.subplot(322),plt.imshow(img_fakeB, cmap = 'gray'), plt.colorbar()
plt.title('Fake_B image'), plt.xticks([]), plt.yticks([])
plt.subplot(323),plt.imshow(img_realB, cmap = 'gray')
plt.title('Real_B image'), plt.xticks([]), plt.yticks([])
plt.subplot(324),plt.imshow(img_realA, cmap = 'gray')
plt.title('Real_A image'), plt.xticks([]), plt.yticks([])
plt.subplot(325),plt.imshow(diff_farb, cmap = 'RdBu'), plt.colorbar()
plt.title('fakeA-realB'), plt.xticks([]), plt.yticks([])
plt.subplot(326),plt.imshow(diff_fbra, cmap = 'RdBu'), plt.colorbar()
plt.title('fakeB-realA'), plt.xticks([]), plt.yticks([])
#
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, axes = plt.subplots(2,2, figsize=(10,10))

# the colormaps we'll be trying out
cmap_list = ['RdPu', 'spring', 'PRGn', 'gnuplot']

for ax, name in zip(axes.flatten(), cmap_list):
    im = ax.imshow(img_fakeA, aspect='auto',  cmap=plt.get_cmap(name))
    ax.yaxis.set_major_locator(plt.NullLocator())   # remove y axis ticks
    ax.xaxis.set_major_locator(plt.NullLocator())   # remove x axis ticks
    ax.set_aspect('equal', adjustable='box')        # make subplots square
    ax.set_title(f'Cmap: {name}', fontsize=18)      # add a title to each
    divider = make_axes_locatable(ax)               # make colorbar same size as each subplot
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax)

In [ ]:
path = "pytorch-CycleGAN-and-pix2pix-master/datasets/w2b/trainA/0a0a1d17-5fb4ffcc-a8ca1541-8c44d583-fd9c6388.jpg"
img = cv2.imread(path)

In [ ]:
img.dtype

In [ ]:
x=img.ravel() >250

In [ ]:
x=x[x==True]

In [ ]:
len(x)

In [ ]:
input_prediction = multilabel_predict_test
input_df = combined
input_prediction_df = pd.DataFrame(input_prediction)
true_logits = pd.DataFrame()
loss_log = pd.DataFrame()

In [ ]:
from sklearn.metrics import average_precision_score

In [ ]:
def stat_calc(input_prediction_df, input_df):
    ground_truth = input_df.race
    pathology_array=['ASIAN',
        'BLACK/AFRICAN AMERICAN',
        'WHITE'
        ]
    i=0
    auc_array = []
    for pathology in pathology_array:
    
        new_truth = (ground_truth.str.contains(pathology)).apply(int)
        input_prediction_val = input_prediction_df[i]
        val = input_prediction_val
        AUC = roc_auc_score(new_truth, val)
        #AUC = average_precision_score(new_truth, val, average=None)
        true_logits.insert(i, i, new_truth, True)
        auc_array.append(AUC)
        i += 1
        
    progress_df = pd.DataFrame({'Study':pathology_array, 'AUC':auc_array})
    print(progress_df)

In [ ]:
stat_calc(input_prediction_df, input_df)

In [ ]:
!pip install seaborn

In [ ]:
import os
import sys
import glob
#import h5py
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm, trange
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import auc, accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

In [ ]:
result = multilabel_predict_test
#result = model.predict(validate_batches, val_epoch)
labels = np.argmax(result, axis=1)
target_names = ['Asian','Black', 'White']

print ('Classwise ROC AUC \n')
for p in list(set(labels)):
    fpr, tpr, thresholds = roc_curve(test_batches.classes, result[:57408][:,p], pos_label = p)
    auroc = round(auc(fpr, tpr), 2)
    print ('Class - {} ROC-AUC- {}'.format(target_names[p], auroc))

print (classification_report(test_batches.classes, labels[:57408], target_names=target_names))
class_matrix = confusion_matrix(test_batches.classes, labels[:57408])

#sns.heatmap(class_matrix, annot=True, fmt='d', cmap='Blues')
heat=sns.heatmap(class_matrix, annot=True, fmt='d', cmap='Blues',xticklabels=target_names,yticklabels=target_names)
heat.set_xlabel("Predicted Label")
heat.set_ylabel("True Label")
plt.show()